## 1. Packages

In [ ]:
using Agents
using GLMakie
using Distributions
using Measures
using Statistics
using Random

## 2. Agent definition

In [2]:
@agent struct SchellingAgent(GridAgent{2}) # Agent on a 2D grid (adapted from Agents.jl)
    mood::Bool
    group::Int
end

## 3. Model initialization 

In [3]:

function initialize(; M::Int=10,      # Grid size (MxM)
    min_to_be_happy::Float64=0.5,     # Minimum fraction of similar neighbors to be happy
    ρ::Float64=0.9,                   # Density of agents on grid
    proportion_red::Float64=0.5,      # Proportion of red agents (group 1)
    lonely_agents_unhappy::Bool=true) # Whether agents with no neighbors are unhappy


    gridsize = (M, M)
    space = GridSpaceSingle(gridsize; periodic=true, metric=:chebyshev) # Toroidal grid # PARAM
    total_agents = round(Int, ρ * prod(gridsize))  # Calculate number of agents based on density

    properties = Dict(
        :min_to_be_happy => min_to_be_happy,
        :M => M,
        :ρ => ρ,
        :total_agents => total_agents,
        :lonely_agents_unhappy => lonely_agents_unhappy,
        :average_similarity => 0.0,
    )

    # Build model
    model = StandardABM(
        SchellingAgent, space;
        agent_step!,                    # Agent stepping function
        model_step!,                    # Model stepping function
        properties=properties,
        scheduler=Schedulers.Randomly() # Random agent activation order
    )

    # Add agents to random positions on the grid
    for _ in 1:total_agents
        add_agent_single!(
            model;
            mood=false,                            # All agents start unhappy
            group=rand() < proportion_red ? 1 : 2  # Randomly assign to group 1 (red) or 2 (blue)
        )
    end

    return model
end


initialize (generic function with 1 method)

## 4. Dynamics - agent_step! - model_step!

In [4]:

# Define the stepping function for each agent
function agent_step!(agent::SchellingAgent, model)
    minhappy = model.min_to_be_happy
    nbrs = collect(nearby_agents(agent, model))  # Get all neighboring agents

    # Handle case where agent has no neighbors
    if isempty(nbrs)
        if model.lonely_agents_unhappy
            agent.mood = false               # Lonely agents are unhappy
            move_agent_single!(agent, model) # Move to a random empty cell
        else
            agent.mood = true                # Lonely agents are content
        end
        return
    end

    # Calculate fraction of neighbors that are same group
    same = count(n -> n.group == agent.group, nbrs)
    frac = same / length(nbrs)

    # Update mood and move if unhappy
    if frac ≥ minhappy # \geq + tab =  ≥
        agent.mood = true                 # Agent is happy with neighbors
    else
        agent.mood = false                # Agent is unhappy
        move_agent_single!(agent, model)  # Move to random empty cell
    end
end

function model_step!(model)
    # Calculate average similarity of agents with their neighbors
    model.average_similarity = calculate_average_similarity(model)
end



model_step! (generic function with 1 method)

## 5. Helper - measurement



In [5]:
# Calculate the average similarity across all agents with neighbors
function calculate_average_similarity(model)
    sims = Float64[]
    for a in allagents(model)
        nbrs = collect(nearby_agents(a, model)) # Get neighbors
        if isempty(nbrs)
            push!(sims, 1.0) # If no neighbors, consider similarity as 1 (or could be 0 based on interpretation)
        else
            same = count(n -> n.group == a.group, nbrs) # Count similar neighbors
            push!(sims, same / length(nbrs))  # Add fraction of similar neighbors
        end
    end
    return isempty(sims) ? 0.0 : mean(sims)  # Return mean similarity or 0 if no data
end

calculate_average_similarity (generic function with 1 method)

## 6. GUI setup



In [ ]:

# Define color mapping for visualization
function set_color(a::SchellingAgent)
    return a.group == 1 ? :red : :blue  # Group 1 is red, Group 2 is blue
end

# Set up GUI 
function setup_gui(; lonely_agents_unhappy::Bool=true, M=60, ρ=0.8, proportion_red=0.5)

    # Parameter slider for min_to_be_happy threshold
    params = Dict(:min_to_be_happy => 0.1:0.01:0.9)


    fig, _ = abmexploration(
        initialize(M=M, ρ=ρ, min_to_be_happy=0.1,
            proportion_red=0.5,
            lonely_agents_unhappy=lonely_agents_unhappy);
        agent_color=set_color,           # Color agents by group
        mdata=[(:average_similarity)],   # Dynamic plot of the average similarity
        mlabels=["Average Similarity"],  # Label for the dynamic plot
        params
    )
    return fig
end

setup_gui (generic function with 1 method)

## 7. Launch GUI & Controls

In [7]:
# Run GUI 
fig_interactive = setup_gui(; lonely_agents_unhappy=true, M=60, ρ=0.8, proportion_red=0.5) # PARAM
display(fig_interactive)

GLMakie.Screen(...)